# Проект Исследование надежности заемщиков.

### Задача
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку. 

### Шаг 1. Обзор данных

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data.csv')
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


#### Необходимо заполнить пропуски, провести анализ дубликатов и привести к некоторые поля к нужным типам, чтобы продолжать исследование

### Шаг 2.1 Заполнение пропусков

In [4]:
print('Пропущенные значения в столбцах total_income и days_employed')

print(f'Доля пропущенных значений в обоих столбцах:{(data["days_employed"].isnull().sum()/data["children"].count()):.3}')

Пропущенные значения в столбцах total_income и days_employed
Доля пропущенных значений в обоих столбцах:0.101


In [5]:
# Заполним значения медианными, как наиболее показательными
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
data['total_income'] = data['total_income'].fillna(data['total_income'].median())

Причина: Если человек не работал ни одного дня, он не будет вносить это значение, 
соответственно общий доход при 0 рабочих дней равен нулю. Либо это просто техническая ошибка.

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [6]:
data['days_employed'] = abs(data['days_employed'])
#Избавляемся от отрицательных значений дней стажа
#display(data['days_employed'])


In [7]:
data['children'] = abs(data['children'])
data['children'] = data['children'].replace(20,2)
data['children'].unique() 
#Избавляемся от отрицательных и аномальных значений у детей

array([1, 0, 3, 2, 4, 5], dtype=int64)

In [8]:
data['gender'] = data['gender'].replace('XNA','F')
data['gender'].unique()
#Существуют только два гендера!

array(['F', 'M'], dtype=object)

In [9]:
print(data[data['dob_years']<18]['dob_years'].count())
#Если мне не изменяет память, то в России несовершеннолетним кредиты не дают
data = data.loc[data['dob_years'] >= 18]

101


In [10]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21424.000000,21424.000000,21424.000000,21424.000000,21424.000000,21424.000000,2.142400e+04
mean,0.479649,60266.292250,43.497479,0.817914,0.971294,0.080891,1.651979e+05
std,0.755293,133292.330795,12.246934,0.548224,1.419737,0.272673,9.797597e+04
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1025.650954,33.000000,1.000000,0.000000,0.000000,1.078249e+05
50%,0.000000,1810.868889,43.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,4779.464934,53.000000,1.000000,1.000000,0.000000,1.955086e+05
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Шаг 2.3. Изменение типов данных.

In [11]:
#Данные поля лучше разумно привести к целочисленным значениям
data['total_income'] = data['total_income'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')
#display(data['total_income'].head(15))

### Шаг 2.4. Удаление дубликатов.

In [12]:
data = data.drop_duplicates().reset_index(drop=True)
print(data.duplicated().sum())
#Избавился от явных дубликатов
data['education'] = data['education'].str.lower()
data['education'].unique()
#Избавился от неявных дубликатов в education

0


array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Явные дубликаты удаляются при помощи встроенных инструментов pandas, неявные элементы убираются вручную путем приведения их к единому виду.
Дубликаты появляются из за отстутствие валидации при заполнении данных.
Неявные дубликаты имеются в столбце цель(purpose). Но исправлять их будем позже.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

#### Необходимо сформировать дополнительные датафреймы для решения проблемы категориальных переменных образования и семейного положения

In [13]:
df_ed = data[['education_id', 'education']]
df_ed = df_ed.drop_duplicates().reset_index(drop=True)
df_fs = data[['family_status_id', 'family_status']]
df_fs = df_fs.drop_duplicates().reset_index(drop=True)

In [14]:
data = data.drop(columns = ['family_status', 'education'], axis = 1)

In [15]:
display(df_ed)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [16]:
display(df_fs)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


### Шаг 2.6. Категоризация дохода.

In [17]:
# Разобьем людей по показателям дохода на пять категорий исходя из требований заказчика
def mkic(row):
    if row['total_income'] <= 30000:
        cat = 'E'
    elif row['total_income']>=30001 and row['total_income']<=50000:
        cat = 'D'
    elif row['total_income']>=50001 and row['total_income']<=200000:
        cat = 'C'
    elif row['total_income']>=200001 and row['total_income']<=1000000:
        cat = 'B'
    elif row['total_income']>=1000001:
        cat = 'A'
    return cat
data['total_income_category'] = data.apply(mkic, axis=1)
display(data.head(15))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Шаг 2.7. Категоризация целей кредита.

In [18]:
# Избавляюсь от неявных дубликатов в целях кредита
data['purpose'].unique()
d = {'жиль':'операции с недвижимостью', 
     'недвижимо':'операции с недвижимостью',
     'образов':'получение образования',
     'автомоб':'операции с автомобилем',
     'свадьб':'проведение свадьбы'}
def mkic(row):
    for k,v in d.items():
        if k in row['purpose']:
            return v
data['purpose_category'] = data.apply(mkic, axis=1)

In [19]:
print(data['purpose_category'].value_counts())
data = data.drop(columns = 'purpose', axis = 1)
display(data.head())

операции с недвижимостью    10767
операции с автомобилем       4286
получение образования        3996
проведение свадьбы           2321
Name: purpose_category, dtype: int64


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,B,получение образования
4,0,340266,53,1,1,F,пенсионер,0,158616,C,проведение свадьбы



### Ответы на вопросы.

### Вопрос 1:
Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [20]:
table = pd.pivot_table(data, values='income_type', index=['children'],
                    columns='debt', aggfunc='count', fill_value=0)
table['Процент возврата'] = round(table[0]/(table[0]+table[1])*100,2)
table = table.rename(  columns={  0: 'Вернули', 1:'Не вернули','children':'Количество детей'})
display(table.head(10))

debt,Вернули,Не вернули,Процент возврата
children,,,
0,12980,1058,92.46
1,4398,442,90.87
2,1912,202,90.44
3,301,27,91.77
4,37,4,90.24
5,9,0,100.00


In [21]:
table = pd.pivot_table(data, values='debt', index=['purpose_category'],
                    columns='children', aggfunc='mean', fill_value=0)
display(table.head(10))

children,0,1,2,3,4,5
purpose_category,,,,,,
операции с автомобилем,0.085392,0.105481,0.121065,0.083333,0.100000,0
операции с недвижимостью,0.067263,0.080697,0.084826,0.077844,0.142857,0
получение образования,0.087139,0.102857,0.112981,0.057971,0.000000,0
проведение свадьбы,0.073905,0.095865,0.066964,0.156250,0.000000,0


##### Вывод 1:
Люди с 5 детьми всегда возвращают кредиты, в остальном зависимость между  количеством детей и возвратом кредита в срок не подтвержена. Все категории возвращают кредиты в одних и тех же интервалах. 

##### Вопрос 2:
Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [22]:
table = pd.pivot_table(data, values='debt', index=['family_status_id'],
                    columns='purpose_category', aggfunc='mean', fill_value=0)
table = df_fs.merge(table, on='family_status_id', how='left')
table = table.drop(columns = ['family_status_id'], axis = 1)
display(table.head(10))

,family_status,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
0,женат / замужем,0.083272,0.069311,0.083527,0.000000
1,гражданский брак,0.118056,0.092292,0.148883,0.079276
2,вдовец / вдова,0.087963,0.051756,0.076142,0.000000
3,в разводе,0.075540,0.070149,0.071730,0.000000
4,Не женат / не замужем,0.129338,0.081285,0.108202,0.000000


##### Вывод 2:
Зависимость между семейным положением и возвратом кредита в срок не подтверждена. Люди живущие в гражданском браке иногда не возвращают кредит за свадьбу.

##### Вопрос 3:
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [23]:
table = pd.pivot_table(data, values='debt', index=['total_income_category'],
                    columns='purpose_category', aggfunc='mean', fill_value=0)
display(table.head(10))

purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
total_income_category,,,,
A,0.000000,0.058824,0.250000,0.000000
B,0.083658,0.066954,0.078652,0.051565
C,0.095507,0.074707,0.098312,0.088032
D,0.140845,0.035928,0.026316,0.088235
E,0.000000,0.181818,0.000000,0.000000


##### Вывод 3:
Чаще всего возвращают кредит люди с категорией дохода D в 94% случаев, а вот люди с категорией E в 1 из 10 кредитов не возвращают долг. Люди с категории дохода А в 1 из 4 случаев не возращают кредит, если берут его на образование.

##### Вопрос 4:
Как разные цели кредита влияют на его возврат в срок?

In [24]:
for i in data['purpose_category'].unique():
    f1 = data[data['purpose_category']==i]
    fd1 = f1[f1['debt']==0]
    print('Процент возрата кредита на',i,'равен:',round(fd1['debt'].count()/f1['debt'].count()*100,2))

Процент возрата кредита на операции с недвижимостью равен: 92.76
Процент возрата кредита на операции с автомобилем равен: 90.67
Процент возрата кредита на получение образования равен: 90.74
Процент возрата кредита на проведение свадьбы равен: 92.07


In [25]:
table = pd.pivot_table(data, values='income_type', index=['purpose_category'],
                    columns='debt', aggfunc='count', fill_value=0)
table['Процент возврата'] = round(table[0]/(table[0]+table[1])*100,2)
table = table.rename(  columns={  0: 'Вернули', 1:'Не вернули','children':'Категория'})
display(table.head(10))

debt,Вернули,Не вернули,Процент возврата
purpose_category,,,
операции с автомобилем,3886,400,90.67
операции с недвижимостью,9988,779,92.76
получение образования,3626,370,90.74
проведение свадьбы,2137,184,92.07


##### Вывод 4:
Зависимость между целью кредита и возвратом кредита в срок не подтверждена.

## Общий вывод:

Явных зависимостей влияющих на возврат кредита в срок не выявлено. В каждой категории присутствует 6-10% людей, которые не возвращают кредит в срок. Исключение составляют люди с 5 детьми, которые всегда возвращают проект в срок.

### P.S. Продолжение следует...
В будущем прикручу ML в виде классификатора, которая будет определять способность клиентов отдавать кредит